cfg => dictonary => single nn.Module => connect nn.Module to an network <br>

In [54]:
from __future__ import division

import import_ipynb
import torch 
import torch.nn as nn
import torch.nn.functional as F 
import numpy as np
import cv2  
from util import *

Dictonary aus cfg Datei erzeugen

In [55]:
def construct_cfg(configFile):
    '''
    Build the network blocks using the configuration file.
    Pre-process it to form easy to manupulate using pytorch.
    ''' 
    
    # Read and pre-process the configuration file
    with open(configFile,'r') as config:
        file = config.read().split('\n')

        file = [line for line in file if len(line) > 0 and line[0]!= '#']
        file = [line.lstrip().rstrip() for line in file]

        #Separate network blocks in a list 

        networkBlocks = [] 
        networkBlock = {}

        for x in file:
            if x[0] == '[':
                if len(networkBlock) != 0:
                    networkBlocks.append(networkBlock)
                    networkBlock = {}
                networkBlock["type"] = x[1:-1].rstrip()
            else:
                entity , value = x.split('=')
                networkBlock[entity.rstrip()] = value.lstrip()
        networkBlocks.append(networkBlock)
    
    return networkBlocks

In [56]:
### Test block ###
# dict_blocks = construct_cfg('C:/Users/User/Yolov4/cfg/yolov3.cfg')
# print(dict_blocks)
# print(dict_blocks[0]['width'])

In [57]:
# dn_info = dict_blocks[0]
# modules = nn.ModuleList([])
# channels = int(dict_blocks[0]['channels'])
# filter_tracker = [] # number filters for each layer

# for j in range(255):
#     filter_tracker.append(j)

# for i, x in enumerate(dict_blocks[1:]):
#     seqModule  = nn.Sequential()

#     typ = x["type"]
    
#     if typ == "route":
#         layer = x['layers']
#         layer = layer.split(',')
#         layer = [int(layer[i]) for i in range(len(layer))]
        
#         filters = 0
#         for single_layer in layer:
#             if single_layer > 0:
#                 filters += filter_tracker[single_layer]
#             else:
#                 filters += filter_tracker[i+single_layer]
        
#         route = EmptyLayer()
#         seqModule.add_module("route_{0}".format(i),route)
        
#         print(i,layer, filters)
#         print(i, type(layer[0]))
#         print(i, layer[0])
#         print(seqModule)

All the diffrent network types and activation functions

In [58]:
### Shows diffrent network types and activation functions ###
# dict_blocks = construct_cfg('C:/Users/User/Yolov4/cfg/yolov4.cfg')
# type_list = []
# activation_list = []
# for i in range(len(dict_blocks)):
# #     print(dict_blocks[i]['type'])
#     typ = dict_blocks[i]['type']
#     if (typ in type_list) == False:
#         type_list.append(typ)
    
#     try:
#         activation = dict_blocks[i]['activation']
#         if (activation in activation_list) == False:
#             activation_list.append(activation)
#     except:
#         pass
    
# print('network types:\n', type_list)
# print('activation functions:\n',activation_list)
# print('maxpool:\n',dict_blocks[109])
# print('route:\n', dict_blocks[114])
# print(len(dict_blocks[114]["layers"].split(',')))
# a = dict_blocks[114]['layers'].split(',')
# print('yolo:\n', dict_blocks[162])

net is just information about the network <br>
convolutional, maxpool and upsample already exist in the pytroch libary. <br>
route, shortcut and yolo must be added 

Neue Klassen <br>
Mish <br>
EmptyLayer <br>
Maxpool <br>

In [59]:
class Mish(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self,x):
        x = x * (torch.tanh(torch.nn.functional.softplus(x)))
        return x
        
class EmptyLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
class Detector(nn.Module):
    def __init__(self, anchors, number_classes, image_size):
        super().__init__()
        self.anchors = anchors
        self.number_classes = number_classes
        self.image_size = image_size

In [60]:
def build_single_blocks(networkBlocks):
    dn_info = networkBlocks[0]
    modules = nn.ModuleList([])
    channels = int(networkBlocks[0]['channels'])
    filter_tracker = [] # number filters for each layer
    
    for i, x in enumerate(networkBlocks[1:]):
        seqModule  = nn.Sequential()
        
        typ = x["type"]
           
        if typ == "convolutional":
#             print(i, typ)
            activation = x["activation"]
            filters = int(x["filters"])
            pad = int(x["pad"])
            kernel_size = int(x["size"])
            stride = int(x["stride"])
            
            if pad:
                padding = (kernel_size - 1) // 2  # 10/3 = 3.33333, 10//3 = 3
            else:
                padding = 0
            
            
            
            try: # batch_normalize is not always defined, after batch normalize no bias is needed
                bn = int(x["batch_normalize"])
                bias = False
            except:
                bn = 0
                bias = True  
                

            conv = nn.Conv2d(channels, filters, kernel_size, stride, padding, bias=bias)
            seqModule.add_module("conv_{0}".format(i), conv)    # {0} will be replace with i 
            

        
            if bn:
                bn = nn.BatchNorm2d(filters)
                seqModule.add_module("batch_norm_{0}".format(i), bn)               
                
            if activation == "mish":
                activn = Mish()
                seqModule.add_module("mish_{0}".format(i), activn)
                
            elif activation == "leaky":   
                activn = nn.LeakyReLU(0.1, inplace = True)
                seqModule.add_module("leaky_{0}".format(i), activn)           
            
            
        elif typ == "upsample":
            upsample = nn.Upsample(scale_factor = 2, mode = "bilinear")
            seqModule.add_module("upsample_{}".format(i), upsample)        
#             print(i, typ)

        
        elif typ == "route":


            layer = x['layers']
            layer = layer.split(',')
            layer = [int(layer[i]) for i in range(len(layer))]

            filters = 0
            for single_layer in layer:
                if single_layer > 0:
                    filters += filter_tracker[single_layer]
                else:
                    filters += filter_tracker[i+single_layer]

            route = EmptyLayer()
            seqModule.add_module("route_{0}".format(i),route)

#             print(i, typ, layer)
#             print(i, type(layer[0]))
#             print(i, layer[0])
#             print(seqModule)           
          
            
            
            
        elif typ == "shortcut":
#             print(i, typ)
            shortcut = EmptyLayer()
            seqModule.add_module("shortcut_{0}".format(i),shortcut)
            
            
        elif typ == "maxpool":
#             print(i, typ)
            stride = int(x["stride"])
            kernel_size = int(x["size"])
            padding  = (kernel_size - 1) // 2
            
            maxpool = nn.MaxPool2d(kernel_size, stride, padding, dilation=1, return_indices=False, ceil_mode=False)
            seqModule.add_module("maxpool_{}".format(i), maxpool) 
            
        elif typ == "yolo":
#             print(i, typ)
            anchors = x["anchors"].split(',')
            anchors = [int(a) for a in anchors]
            masks = x["mask"].split(',')
            masks = [int(a) for a in masks]
            anchors = [(anchors[j],anchors[j+1]) for j in range(0,len(anchors),2)]
            anchors = [anchors[j] for j in masks]
            number_classes = x['classes']
            image_size = dn_info['width']
#             print(number_classes,image_size)
            detector_layer = Detector(anchors, number_classes, image_size)
            seqModule.add_module("Detection_{0}".format(i), detector_layer)
           
        modules.append(seqModule)
#         print(i, channels,filters)
        channels = filters
        filter_tracker.append(filters)
        
#         print('filters: ', filters)
#         print(filters)
#     print(filter_tracker)
#     print(filter_tracker)
    return (dn_info, modules)


In [61]:
# ### Test ###
# dict_blocks = construct_cfg('C:/Users/User/Yolov4/cfg/yolov3.cfg')
# dn_info, module = build_single_blocks(dict_blocks)
# print(module)
# print(dict_blocks)
# print(dn_info)
# print(dict_blocks[1:])
# for i in range(len(dict_blocks)-1):
#     try:
#         print(dict_blocks[i+1]["filters"])
#     except:
#         try:
#             print('route: ',dict_blocks[i+1]["layers"])
#         except:
#             print('shortcuts: ',dict_blocks[i+1]["from"])

# for i in range(len(dict_blocks)-1):
#     print(module[i])
# # print(dict_blocks[10])
# # print(dict_blocks[9])
# # print(dict_blocks[3])

In [62]:
# print(len(module))
# for i in range(len(module)):
#     network_blocks = construct_cfg('C:/Users/User/Yolov4/cfg/yolov4.cfg')
#     module = network_blocks[1:]
#     module_type = (module[i]["type"])
#     layer_outputs = {}
#     for k in range(161):
#         layer_outputs[k] = k
#     print(layer_outputs)
#     if module_type == "route":
#         module_layer = module[i]["layers"]
#         print(module_layer)
#         print(type(module_layer[0]))
#         module_layer = module_layer.split(',')
#         module_layer = [int(tmp) for tmp in module_layer]
#         print(type(module_layer[0]))
        
#         tmp_layer = []
#         for j in range(len(module_layer)):
#             if module_layer[j] > 0:
#                 tmp_layer.append(layer_outputs[module_layer[j]])
#             else:
#                 tmp_layer.append(layer_outputs[i+module_layer[j]])
#             tmp_layer_tuple = tuple(tmp_layer)
#             x = torch.cat(tmp_layer_tuple,1)
#         layer_ouputs[i] = x

In [63]:
class Network(nn.Module):
    def __init__(self, cfgfile):
        super().__init__()
        self.network_blocks = construct_cfg(cfgfile) # cfg => dictonary
        self.dn_info, self.module_list = build_single_blocks(self.network_blocks) # dictonary => single nn.Module
        self.header = torch.IntTensor([0,0,0,0])
        self.seen = 0
    
    def forward(self, x, CUDA): # CUDA is bool Ture or False
        detections = []
        modules = self.network_blocks[1:]
        layer_outputs = {} 

        written_output = 0
        #Iterate throught each module 
        for i in range(len(modules)):
#             print(i)
#             print()
            module_type = (modules[i]["type"])
            #Upsampling is basically a form of convolution
            if module_type == "convolutional" or module_type == "upsample" :
#                 print(i)
#                 print(self.module_list)
#                 print(self.module_list[i])
#                 print(self.module_list[i](x))
#                 if i == 10:
#                     print('i ist jetzt gleich 10:')
#                     print(self.module_list)
#                     print(self.module_list[i])
#                     print(self.module_list[i](x))
        
                x = self.module_list[i](x)
                layer_outputs[i] = x
#                 print(i, layer_outputs[i].size())
           
            elif module_type == "maxpool":
                x = self.module_list[i](x)
                layer_outputs[i] = x
    
    
            elif module_type == "route":
                module_layer = modules[i]["layers"]
#                 print(module_layer)
#                 print(type(module_layer[0]))
                module_layer = module_layer.split(',')
                module_layer = [int(tmp) for tmp in module_layer]
#                 print(type(module_layer[0]))

                tmp_layer = []
                for j in range(len(module_layer)):
                    if module_layer[j] > 0:
                        tmp_layer.append(layer_outputs[module_layer[j]])
                    else:
                        tmp_layer.append(layer_outputs[i+module_layer[j]])
                    tmp_layer_tuple = tuple(tmp_layer)
                    x = torch.cat(tmp_layer_tuple,1)
                layer_outputs[i] = x    
                
                

            elif  module_type == "shortcut":
                from_ = int(modules[i]["from"])
                x = layer_outputs[i-1] + layer_outputs[i+from_]
                layer_outputs[i] = x
             
            
            elif module_type == 'yolo':        
                
                anchors = self.module_list[i][0].anchors
#                 print('anchors: ', anchors)
                
                #Get the input dimensions
#                 inp_dim = int(self.dn_info["height"])
                inp_dim = int(self.module_list[i][0].image_size)
#                 print('inp_dim: ', inp_dim)
                
                #Get the number of classes
#                 num_classes = int(modules[i]["classes"])
                num_classes = int(self.module_list[i][0].number_classes)
#                 print('num_classes: ', num_classes)
                
                #Output the result
                x = x.data
                print("Size before transform => " ,x.size())
                
                
                #Convert the output to 2D (batch x grids x bounding box attributes)
                x = transformOutput(x, inp_dim, anchors, num_classes, CUDA)
                print("Size after transform => " ,x.size())

                
                #If no detections were made
                if type(x) == int:
                    continue

                
                if not written_output:
                    detections = x
                    written_output = 1
                
                else:
                    detections = torch.cat((detections, x), 1)
                
                layer_outputs[i] = layer_outputs[i-1]  
                  

        try:
            return detections
        except:
            return 0 
        
################################################ Not totaly clear #########################
    def load_weights(self, weightfile):
        with open(weightfile, "rb") as fp:

            #The first 4 values are header information 
            # 1. Major version number
            # 2. Minor Version Number
            # 3. Subversion number 
            # 4. IMages seen 
            header = np.fromfile(fp, dtype = np.int32, count = 5)
            self.header = torch.from_numpy(header)
            self.seen = self.header[3]


            weights = np.fromfile(fp, dtype = np.float32)

            tracker = 0
            for i in range(len(self.module_list)):
                module_type = self.network_blocks[i + 1]["type"]
                if module_type == "convolutional":
                    model = self.module_list[i]
                    try:
                        batch_normalize = int(self.network_blocks[i+1]["batch_normalize"])
                    except:
                        batch_normalize = 0

                    convPart = model[0]

                    if (batch_normalize):
                        #Weights file Configuration=> bn bais->bn weights-> running mean-> running var
                        #The weights are arranged in the above mentioned order
                        bnPart = model[1]

                        biasCount = bnPart.bias.numel()

                        bnBias = torch.from_numpy(weights[tracker:tracker + biasCount])
                        tracker += biasCount

                        bnPart_weights = torch.from_numpy(weights[tracker: tracker + biasCount])
                        tracker  += biasCount

                        bnPart_running_mean = torch.from_numpy(weights[tracker: tracker + biasCount])
                        tracker  += biasCount

                        bnPart_running_var = torch.from_numpy(weights[tracker: tracker + biasCount])
                        tracker  += biasCount

                        bnBias = bnBias.view_as(bnPart.bias.data)
                        bnPart_weights = bnPart_weights.view_as(bnPart.weight.data)
                        bnPart_running_mean = bnPart_running_mean.view_as(bnPart.running_mean)
                        bnPart_running_var = bnPart_running_var.view_as(bnPart.running_var)

                        bnPart.bias.data.copy_(bnBias)
                        bnPart.weight.data.copy_(bnPart_weights)
                        bnPart.running_mean.copy_(bnPart_running_mean)
                        bnPart.running_var.copy_(bnPart_running_var)

                    else:
                        biasCount = convPart.bias.numel()

                        convBias = torch.from_numpy(weights[tracker: tracker + biasCount])
                        tracker = tracker + biasCount

                        convBias = convBias.view_as(convPart.bias.data)

                        convPart.bias.data.copy_(convBias)


                    weightfile = convPart.weight.numel()

                    convWeight = torch.from_numpy(weights[tracker:tracker+weightfile])
                    tracker = tracker + weightfile

                    convWeight = convWeight.view_as(convPart.weight.data)
                    convPart.weight.data.copy_(convWeight) 

In [64]:
#################### Test #######################
num_classes = 80
classes = load_classes('data/coco.names') 
model = Network('C:/Users/User/Yolov4/cfg/yolov3.cfg')
test_data = torch.randn(1,3,256,256,dtype = torch.float)
test_output = model(test_data, False)

Size before transform =>  torch.Size([1, 255, 8, 8])
Size after transform =>  torch.Size([1, 192, 85])
Size before transform =>  torch.Size([1, 255, 16, 16])
Size after transform =>  torch.Size([1, 768, 85])
Size before transform =>  torch.Size([1, 255, 32, 32])
Size after transform =>  torch.Size([1, 3072, 85])


C:\Users\User\Anaconda3\lib\site-packages\torch\nn\functional.py:3121: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [65]:
print(model.module_list[82][0].anchors)
print(model.module_list[82][0].anchors)
model.load_weights("C:/Users/User/weights/yolov3.weights")
print("Network loaded")

[(116, 90), (156, 198), (373, 326)]
Network loaded


In [16]:
file = 'model_yolov3.pth'
torch.save(model.state_dict(), file) #model.state_dict() speichert die Parameter

In [150]:
# load
loaded_model = Network('C:/Users/User/Yolov4/cfg/yolov3.cfg')
loaded_model.load_state_dict(torch.load(file))
loaded_model.eval()

Network(
  (module_list): ModuleList(
    (0): Sequential(
      (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_0): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (1): Sequential(
      (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_1): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (2): Sequential(
      (conv_2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (batch_norm_2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_2): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (3): Sequential(
      (conv_3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch

https://github.com/eriklindernoren/PyTorch-YOLOv3/issues/133

In [223]:
### Einlesen der neuen Parameter ### am besten aus einem config file
model = Network('C:/Users/User/Yolov4/cfg/yolov3.cfg')
number_classes = 70
image_size = 1024
anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
unfreeze_layer = 0 # Anzahl layer vor yolo layer wo die Gradienten True gesetzt werden. Wert muss negativ sein und mindesten -1 betragen.


bb_attrib_count = (5 + number_classes) * 3
k = 0 # für anchor
detector_layers = []
for i in range(len(model.module_list)):
    
    string_module_list = str(model.module_list[i][0])
    if 'Detector()' == string_module_list[:10]: # Detector() layer suchen
#         print(i)
        detector_layers.append(i)
    
        ### yolo layer anpassen, neue anchor, Klassenanzahl, Bildeingangsgrösse ###
        model.module_list[i][0].anchors = [(anchors[k+0],anchors[k+1]),(anchors[k+2],anchors[k+3]),(anchors[k+4],anchors[k+5])]
        model.module_list[i][0].number_classes = int(number_classes)
        model.module_list[i][0].image_size = int(image_size)
#         print('anchors: ', model.module_list[i][0].anchors)
#         print('number_classes: ', model.module_list[i][0].number_classes)
#         print('image_size: ', model.module_list[i][0].image_size)
        
        ### conv layer vor yolo layer anpassen, da Filter und Klassenanzahl ändert ###
        conv_old = model.module_list[i-1][0]
        new_conv = torch.nn.Conv2d(in_channels = conv_old.in_channels,
                                   out_channels = bb_attrib_count,
                                   kernel_size = conv_old.kernel_size,
                                   stride = conv_old.stride,
                                   padding = conv_old.padding)
        
        model.module_list[i-1][0] = new_conv
#         print('conv: ', model.module_list[i-1][0])
        k += 6

### Gradiente einfrieren/auftauen 
for name, param in model.named_parameters():
    # module_list.21.batch_norm_21.bias
    layer_id = int(name.split('.')[-3])
#     print(layer_id)
#     print(detector_layers)
    # Freeze layers before 6
    param.requires_grad = False

    for layer in detector_layers:
        if layer > layer_id and layer_id >= layer+unfreeze_layer: # Gradienten True setzen. Anzahl unfreeze_layer zurück vom yolo_layer
            tmp_grad = True
            break
        else:
            tmp_grad = False

#     print(str(tmp_grad) + ': ', name) # Liste aller Layer, wo man sieht, ob der Gradient True oder False ist
    param.requires_grad = tmp_grad



False:  module_list.0.conv_0.weight
False:  module_list.0.batch_norm_0.weight
False:  module_list.0.batch_norm_0.bias
False:  module_list.1.conv_1.weight
False:  module_list.1.batch_norm_1.weight
False:  module_list.1.batch_norm_1.bias
False:  module_list.2.conv_2.weight
False:  module_list.2.batch_norm_2.weight
False:  module_list.2.batch_norm_2.bias
False:  module_list.3.conv_3.weight
False:  module_list.3.batch_norm_3.weight
False:  module_list.3.batch_norm_3.bias
False:  module_list.5.conv_5.weight
False:  module_list.5.batch_norm_5.weight
False:  module_list.5.batch_norm_5.bias
False:  module_list.6.conv_6.weight
False:  module_list.6.batch_norm_6.weight
False:  module_list.6.batch_norm_6.bias
False:  module_list.7.conv_7.weight
False:  module_list.7.batch_norm_7.weight
False:  module_list.7.batch_norm_7.bias
False:  module_list.9.conv_9.weight
False:  module_list.9.batch_norm_9.weight
False:  module_list.9.batch_norm_9.bias
False:  module_list.10.conv_10.weight
False:  module_lis

80


module_list.0.conv_0.weight
Parameter containing:
tensor([[[[ 1.4825e-01,  1.2516e-01,  1.6522e-04],
          [-1.1882e-01, -9.4695e-02,  1.7941e-02],
          [-5.3494e-02, -6.5786e-02, -3.2635e-02]],

         [[ 5.1022e-02,  1.9926e-02, -1.8118e-01],
          [-6.1785e-02, -2.6592e-02,  1.3691e-01],
          [ 7.5410e-02,  3.3555e-02,  1.2573e-01]],

         [[-9.5002e-02, -9.7676e-02,  1.0462e-01],
          [ 1.6712e-01, -3.0024e-02, -1.4913e-01],
          [-4.3464e-02, -1.5494e-01, -1.0926e-01]]],


        [[[ 4.7231e-02, -1.0478e-01,  1.7150e-01],
          [ 1.8956e-01,  5.0838e-02, -1.8258e-01],
          [-1.0826e-01,  1.6545e-01, -6.4063e-02]],

         [[ 5.8674e-02, -7.9378e-02,  1.1273e-01],
          [-1.6967e-01,  2.8191e-03,  6.8989e-02],
          [ 1.8935e-01,  2.8134e-02, -6.8729e-02]],

         [[ 2.3616e-02,  7.9829e-02, -8.5353e-02],
          [-1.8457e-01, -6.2582e-02,  2.8548e-02],
          [ 1.0299e-01,  1.0860e-01,  8.4544e-02]]],


        [[[-2.76

Parameter containing:
tensor([[[[-0.0156,  0.0143, -0.0018],
          [-0.0263, -0.0232, -0.0016],
          [-0.0041, -0.0505,  0.0489]],

         [[-0.0125,  0.0259,  0.0210],
          [-0.0178, -0.0414, -0.0283],
          [ 0.0040, -0.0217, -0.0478]],

         [[-0.0081, -0.0002, -0.0277],
          [ 0.0186,  0.0307, -0.0040],
          [-0.0073,  0.0539,  0.0578]],

         ...,

         [[-0.0331, -0.0170,  0.0373],
          [-0.0195, -0.0225, -0.0444],
          [ 0.0269,  0.0519,  0.0237]],

         [[ 0.0312, -0.0410,  0.0313],
          [-0.0275,  0.0329, -0.0087],
          [-0.0497, -0.0568, -0.0063]],

         [[-0.0086,  0.0040, -0.0158],
          [-0.0075,  0.0077,  0.0578],
          [-0.0188, -0.0186, -0.0561]]],


        [[[ 0.0487,  0.0561, -0.0141],
          [-0.0305,  0.0445,  0.0090],
          [-0.0450, -0.0479, -0.0005]],

         [[ 0.0568, -0.0273, -0.0045],
          [ 0.0279,  0.0273,  0.0394],
          [ 0.0492,  0.0257,  0.0514]],

         

Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.], requires_grad=True)
module_list.22.batch_norm_22.bias
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0

Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1.

Parameter containing:
tensor([1., 1., 1.,  ..., 1., 1., 1.], requires_grad=True)
module_list.64.batch_norm_64.bias
Parameter containing:
tensor([0., 0., 0.,  ..., 0., 0., 0.], requires_grad=True)
module_list.66.conv_66.weight
Parameter containing:
tensor([[[[-0.0023]],

         [[ 0.0123]],

         [[ 0.0279]],

         ...,

         [[-0.0055]],

         [[-0.0214]],

         [[ 0.0236]]],


        [[[ 0.0223]],

         [[-0.0165]],

         [[ 0.0173]],

         ...,

         [[-0.0032]],

         [[-0.0104]],

         [[ 0.0081]]],


        [[[-0.0169]],

         [[ 0.0039]],

         [[-0.0119]],

         ...,

         [[-0.0120]],

         [[ 0.0289]],

         [[ 0.0005]]],


        ...,


        [[[-0.0312]],

         [[ 0.0234]],

         [[ 0.0121]],

         ...,

         [[ 0.0001]],

         [[ 0.0238]],

         [[-0.0146]]],


        [[[-0.0289]],

         [[-0.0158]],

         [[-0.0069]],

         ...,

         [[-0.0119]],

         [

Parameter containing:
tensor([[[[ 0.0080]],

         [[ 0.0086]],

         [[ 0.0092]],

         ...,

         [[-0.0357]],

         [[ 0.0085]],

         [[ 0.0441]]],


        [[[ 0.0080]],

         [[ 0.0333]],

         [[ 0.0183]],

         ...,

         [[-0.0121]],

         [[ 0.0220]],

         [[-0.0127]]],


        [[[-0.0165]],

         [[ 0.0293]],

         [[ 0.0291]],

         ...,

         [[-0.0266]],

         [[-0.0196]],

         [[-0.0303]]],


        ...,


        [[[ 0.0330]],

         [[-0.0289]],

         [[-0.0133]],

         ...,

         [[ 0.0010]],

         [[-0.0058]],

         [[ 0.0147]]],


        [[[ 0.0054]],

         [[ 0.0407]],

         [[-0.0029]],

         ...,

         [[ 0.0060]],

         [[-0.0236]],

         [[ 0.0049]]],


        [[[ 0.0426]],

         [[ 0.0334]],

         [[-0.0163]],

         ...,

         [[ 0.0242]],

         [[ 0.0410]],

         [[-0.0234]]]], requires_grad=True)
module_list.84

Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1.

SyntaxError: invalid syntax (<ipython-input-41-74533c2faa06>, line 18)

0
0
0
1
1
1
2
2
2
3
3
3
5
5
5
6
6
6
7
7
7
9
9
9
10
10
10
12
12
12
13
13
13
14
14
14
16
16
16
17
17
17
19
19
19
20
20
20
22
22
22
23
23
23
25
25
25
26
26
26
28
28
28
29
29
29
31
31
31
32
32
32
34
34
34
35
35
35
37
37
37
38
38
38
39
39
39
41
41
41
42
42
42
44
44
44
45
45
45
47
47
47
48
48
48
50
50
50
51
51
51
53
53
53
54
54
54
56
56
56
57
57
57
59
59
59
60
60
60
62
62
62
63
63
63
64
64
64
66
66
66
67
67
67
69
69
69
70
70
70
72
72
72
73
73
73
75
75
75
76
76
76
77
77
77
78
78
78
79
79
79
80
80
80
81
81
84
84
84
87
87
87
88
88
88
89
89
89
90
90
90
91
91
91
92
92
92
93
93
96
96
96
99
99
99
100
100
100
101
101
101
102
102
102
103
103
103
104
104
104
105
105


[(116, 50), (156, 198), (373, 326)]


[(116, 90), (156, 198), (373, 326)]
